In [1]:
"""
@Description :   How to use the lightglue and xfeat
@Author      :   Xubo Luo 
@Time        :   2024/07/10 20:09:58
"""


from lightglue.lightglue import LightGlue
from lightglue.utils import rbd
from xfeat.xfeat import XFeat

import torch
import cv2

/home/wuaodi/anaconda3/envs/fastmatch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'
matcher = LightGlue(features="superpoint").eval().to(device)
xfeat = XFeat(weights='weights/xfeat.pt')

loading weights from: weights/xfeat.pt


/home/wuaodi/anaconda3/envs/fastmatch/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
/home/wuaodi/workspace/Codes/dev/FastMatch/lightglue/lightglue.py:143: UserWarning: FlashAttention is not available. For optimal speed, consider installing torch >= 2.0 or flash-attn.
  self.inner_attn = Attention(flash)


In [11]:
image0 = cv2.imread("./assets/ref.png")
image1 = cv2.imread("./assets/tgt.png")
img_size = torch.tensor([[image0.shape[1], image0.shape[0]]]).to(device)

# Extract keypoints and descriptors
image0 = xfeat.parse_input(image0)
image1 = xfeat.parse_input(image1)

feat0 = xfeat.detectAndCompute(image0, top_k = 2048)[0]
feat1 = xfeat.detectAndCompute(image1, top_k = 2048)[0]

feat0['keypoints'] = feat0['keypoints'].unsqueeze(0)
feat1['keypoints'] = feat1['keypoints'].unsqueeze(0)
feat0['descriptors'] = feat0['descriptors'].unsqueeze(0)
feat1['descriptors'] = feat1['descriptors'].unsqueeze(0)

(600, 800, 3) (600, 800, 3)
tensor([800, 600])


In [9]:
feat0['keypoint_scores'] = feat0['scores']
feat1['keypoint_scores'] = feat1['scores']

feat0['image_size'] = img_size
feat1['image_size'] = img_size

feat0['descriptors'] = torch.rand(1, 2048, 256).to(device)
feat1['descriptors'] = torch.rand(1, 2048, 256).to(device)

dict_keys(['keypoints', 'scores', 'descriptors', 'keypoint_scores', 'image_size'])


In [10]:
matches01 = matcher({"image0":feat0, "image1":feat1})

feat0, feat1, matches01 = [
    rbd(x) for x in (feat0, feat1, matches01)
]   # remove batch dimension

kpts0, kpts1, matches = feat0["keypoints"], feat1["keypoints"], matches01["matches"]
m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

/home/wuaodi/anaconda3/envs/fastmatch/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
axes = viz2d.plot_images([image0, image1])
viz2d.plot_matches(m_kpts0, m_kpts1, color="lime", lw=0.2)
viz2d.add_text(0, f'Stop after {matches01["stop"]} layers', fs=20)